<a href="https://colab.research.google.com/github/salsaimon/proyecto_recursoshumanos/blob/main/GH_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#### Cargar paquetes siempre al inicio
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import sys ## saber ruta de la que carga paquetes

In [31]:

import sys
import os

path='/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos'

os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

import a_funciones as funciones  ###archivo de funciones propias

In [32]:
conn=sql.connect('base.db')  ### crea base de datos
cur=conn.cursor()  ### para ejecutar consultas en la base de datos

In [20]:
####################################################################################################################
########################  1. Comprender y limpiar datos ##################################################################
####################################################################################################################
########   Verificar lectura correcta de los datos
########   Verificar Datos faltantes (eliminar variables si es necesario) (la imputación está la parte de modelado)
########   Tipos de variables (categoricas/numéricas/fechas)
########   Niveles en categorícas
########   Observaciones por categoría
########   Datos atípicos en numéricas

In [33]:
##leer bases de datos

retiros=pd.read_csv('data/retirement_info.csv')
general=pd.read_csv('data/general_data.csv')
employee_survey=pd.read_csv('data/employee_survey_data.csv')
manager_survey=pd.read_csv('data/manager_survey.csv')

#crear df para cada base
df_retiros= retiros
df_general= general
df_employee= employee_survey
df_manager= manager_survey

###### Verificar lectura correcta de los datos
df_retiros.sort_values(by=['EmployeeID'],ascending=1).head(100)
df_general.sort_values(by=['EmployeeID'],ascending=1).head(100)
df_employee.sort_values(by=['EmployeeID'],ascending=1).head(100)
df_manager.sort_values(by=['EmployeeID'],ascending=1).head(100)


,EmployeeID,JobInvolvement,PerformanceRating,SurveyDate
0,1,3,3,2015-12-31
4410,1,3,3,2016-12-31
1,2,2,4,2015-12-31
2,3,3,3,2015-12-31
4412,3,3,3,2016-12-31
...,...,...,...,...
4461,52,2,3,2016-12-31
51,52,2,3,2015-12-31
4462,53,3,4,2016-12-31
52,53,3,4,2015-12-31


In [34]:

##### resumen con información tablas faltantes y tipos de variables y hacer correcciones
df_retiros.info(verbose=True)
df_general.info(verbose=True)
df_employee.info(verbose=True)
df_manager.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         711 non-null    int64 
 1   Attrition          711 non-null    object
 2   retirementDate     711 non-null    object
 3   retirementType     711 non-null    object
 4   resignationReason  641 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      8820 non-null   int64  
 1   BusinessTravel           8820 non-null   object 
 2   Department               8820 non-null   object 
 3   DistanceFromHome         8820 non-null   int64  
 4   Education                8820 non-null   int64  
 5   EducationField           8

In [35]:
#### Convertir campos a formato fecha
df_retiros["retirementDate"]=pd.to_datetime(df_retiros['retirementDate'], format="%Y-%m-%d")
df_general["InfoDate"]=pd.to_datetime(df_general['InfoDate'], format="%Y-%m-%d")
df_employee["DateSurvey"]=pd.to_datetime(df_employee['DateSurvey'], format="%Y-%m-%d")
df_manager["SurveyDate"]=pd.to_datetime(df_manager['SurveyDate'], format="%Y-%m-%d")

In [11]:
#### convertir a categórica

#general
df_general=df_general.astype({'Education': object,'JobLevel': object,'StockOptionLevel': object})
#employee
df_employee=df_employee.astype({'EnvironmentSatisfaction': object,'JobSatisfaction': object,'WorkLifeBalance': object})
#manager
df_manager=df_manager.astype({'JobInvolvement': object,'PerformanceRating': object})

In [12]:
###Eliminar columnas que no se utilicen
#eliminar over 18 de general
df_general=df_general.drop(["Over18"], axis=1)

In [26]:
### Llevar tablas a base de datos
df_retiros.to_sql("retiros",conn,if_exists="replace")
df_general.to_sql("general",conn,if_exists="replace")
df_employee.to_sql("employee",conn,if_exists="replace")
df_manager.to_sql("manager",conn,if_exists="replace")

cur.execute("Select name from sqlite_master where type='table'") ### consultar bases de datos
cur.fetchall()


[('retiros',), ('general',), ('employee',), ('manager',)]

In [27]:
##### verificar categorías y observaciones ######
#####El número de categorías de una variable influye mucho en la eficiencia y sobre ajuste#
### convertir tabla de base de datos en data frame de pandas y hacer consultas ####

#############################RETIROS

pd.read_sql("""select Attrition,count(*)
                            from retiros
                            group by Attrition""", conn)

#recategorizar 0 y 1
pd.read_sql("""select retirementType,count(*)
                            from retiros
                            group by retirementType""", conn)

#recategorizar, tiene 4 categorias
pd.read_sql("""select resignationReason,count(*)
                            from retiros
                            group by resignationReason""", conn)

#############################GENERAL
#recategorizar, 3 categorias
pd.read_sql("""select BusinessTravel,count(*)
                            from general
                            group by BusinessTravel""", conn)

#recategorizar, 3 categorias
pd.read_sql("""select Department,count(*)
                            from general
                            group by Department""", conn)

#recategorizar, 4 categorias
pd.read_sql("""select Education,count(*)
                            from general
                            group by Education""", conn)

#recategorizar, 5 categorias posible eliminacion
pd.read_sql("""select EducationField,count(*)
                            from general
                            group by EducationField""", conn)

#recategorizar, 5 categorias
pd.read_sql("""select  JobLevel,count(*)
                            from general
                            group by  JobLevel""", conn)

#recategorizar, 8 categorias, agrupar primero y luego categorizar
pd.read_sql("""select  JobRole,count(*)
                            from general
                            group by  JobRole""", conn)

#recategorizar, 3 categorias posible eliminacion
pd.read_sql("""select  MaritalStatus,count(*)
                            from general
                            group by  MaritalStatus""", conn)

#recategorizar, 4 categorias
pd.read_sql("""select  StockOptionLevel,count(*)
                            from general
                            group by  StockOptionLevel""", conn)


#############################EMPLOYEE
#recategorizar, 4 categorias
pd.read_sql("""select  EnvironmentSatisfaction,count(*)
                            from employee
                            group by  EnvironmentSatisfaction""", conn)

#recategorizar, 5 categorias
pd.read_sql("""select  JobSatisfaction,count(*)
                            from employee
                            group by  JobSatisfaction""", conn)

#recategorizar, 5 categorias
pd.read_sql("""select  WorkLifeBalance,count(*)
                            from employee
                            group by  WorkLifeBalance""", conn)


#############################MANAGER

#recategorizar, 4 categorias
pd.read_sql("""select  JobInvolvement,count(*)
                            from manager
                            group by  JobInvolvement""", conn)

#recategorizar, 2 categorias 0y 1
pd.read_sql("""select  PerformanceRating,count(*)
                            from manager
                            group by  PerformanceRating""", conn)




,PerformanceRating,count(*)
0,3,7464
1,4,1356


In [28]:
#### para hacer todos los preprocesamienteos se crea archivo .sql que se ejecuta con la función: ejecutar_sql del archivo funciones.py

funciones.ejecutar_sql('preprocesamientos.sql',cur)

cur.execute("select name from sqlite_master where type='table'")
cur.fetchall()

df=pd.read_sql("select * from base_completa  ",conn)
pd.read_sql("select count(distinct empID2)  from base_completa  ",conn)
df.info()
df.describe(include='all')

FileNotFoundError: [Errno 2] No such file or directory: 'preprocesamientos.sql'